In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.nn import MSELoss
import pandas as pd
from AttentiveFP import get_smiles_dicts, get_smiles_array, num_atom_features, num_bond_features  # assumed to be your actual featurizer
from AttentiveFP import Fingerprint  # assumed to be your attentive FP model

In [2]:
# --- Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = 'AMES'

# --- Dataset ---
class MoleculeDataset(Dataset):
    def __init__(self, smiles_list, targets, feature_dicts):
        self.smiles_list = smiles_list
        self.targets = targets
        self.feature_dicts = feature_dicts
        self.x_atom, self.x_bond, self.x_atom_index, self.x_bond_index, self.x_mask, _ = get_smiles_array(smiles_list, feature_dicts)

    def __len__(self):
        return len(self.smiles_list)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.x_atom[idx], dtype=torch.float32),
            torch.tensor(self.x_bond[idx], dtype=torch.float32),
            torch.tensor(self.x_atom_index[idx], dtype=torch.long),
            torch.tensor(self.x_bond_index[idx], dtype=torch.long),
            torch.tensor(self.x_mask[idx], dtype=torch.float32),
            torch.tensor(self.targets[idx], dtype=torch.float32)
        )

In [4]:
# --- Load data ---
df = pd.read_csv(f"../data/{model_name}.csv")
smiles_list = df["smiles"].tolist()
targets = df["active"].tolist()
feature_dicts = get_smiles_dicts(smiles_list)

# --- Dataloader ---
dataset = MoleculeDataset(smiles_list, targets, feature_dicts)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

KeyError: 'CC(C)[C@@H]1CC[C@H](C)[C@@H]2CC[C@H](C)C[C@@H]12'

In [4]:
# --- Model ---
model = Fingerprint(
    radius=5,
    T=3,
    input_feature_dim=num_atom_features(),
    input_bond_dim=num_bond_features(),
    fingerprint_dim=200,
    output_units_num=1,
    p_dropout=0.1
).to(device)

optimizer = Adam(model.parameters(), lr=1e-3)
loss_fn = MSELoss()

In [5]:
from sklearn.model_selection import train_test_split

# Split SMILES and targets
train_smiles, val_smiles, train_targets, val_targets = train_test_split(
    smiles_list, targets, test_size=0.2, random_state=42
)

train_set = MoleculeDataset(train_smiles, train_targets, feature_dicts)
val_set = MoleculeDataset(val_smiles, val_targets, feature_dicts)

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)


In [6]:
import os

# Folder to save model
os.makedirs("../models", exist_ok=True)
best_loss = float("inf")

# Load existing model
# model.load_state_dict(torch.load("../models/model_AMES.pt"))
# model.load_state_dict(torch.load(f"../models/model_{model_name}.pt"))
# model.to(device)
# model.eval()

best_loss = float("inf")
os.makedirs("../models", exist_ok=True)

for epoch in range(30):
    model.train()
    train_loss = 0

    for atom, bond, atom_deg, bond_deg, mask, target in train_loader:
        atom, bond, atom_deg, bond_deg, mask, target = [
            t.to(device) for t in (atom, bond, atom_deg, bond_deg, mask, target)
        ]
        _, pred, _ = model(atom, bond, atom_deg, bond_deg, mask)
        loss = loss_fn(pred.squeeze(), target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # --- Validation ---
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for atom, bond, atom_deg, bond_deg, mask, target in val_loader:
            atom, bond, atom_deg, bond_deg, mask, target = [
                t.to(device) for t in (atom, bond, atom_deg, bond_deg, mask, target)
            ]
            _, pred, _ = model(atom, bond, atom_deg, bond_deg, mask)
            loss = loss_fn(pred.squeeze(), target)
            val_loss += loss.item()

    print(f"Epoch {epoch+1} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

    # Save best model
    if val_loss < best_loss:
        best_loss = val_loss
        path = "../models/model_AMES.pt"
        path = f"../models/model_{model_name}.pt"
        torch.save(model.state_dict(), path)
        print(f"✅ Saved best model to {path} (val loss: {best_loss:.4f})")



Epoch 1 - Train Loss: 163.0026 - Val Loss: 29.7152
✅ Saved best model to ../models/model_Z50.pt (val loss: 29.7152)
Epoch 2 - Train Loss: 118.6357 - Val Loss: 25.5445
✅ Saved best model to ../models/model_Z50.pt (val loss: 25.5445)
Epoch 3 - Train Loss: 114.7395 - Val Loss: 28.9259
Epoch 4 - Train Loss: 108.3598 - Val Loss: 23.9633
✅ Saved best model to ../models/model_Z50.pt (val loss: 23.9633)
Epoch 5 - Train Loss: 104.8730 - Val Loss: 25.9855
Epoch 6 - Train Loss: 104.4509 - Val Loss: 24.6480
Epoch 7 - Train Loss: 99.7017 - Val Loss: 22.6848
✅ Saved best model to ../models/model_Z50.pt (val loss: 22.6848)
Epoch 8 - Train Loss: 96.1251 - Val Loss: 22.0022
✅ Saved best model to ../models/model_Z50.pt (val loss: 22.0022)
Epoch 9 - Train Loss: 93.6719 - Val Loss: 21.7462
✅ Saved best model to ../models/model_Z50.pt (val loss: 21.7462)
Epoch 10 - Train Loss: 89.9997 - Val Loss: 20.2040
✅ Saved best model to ../models/model_Z50.pt (val loss: 20.2040)
Epoch 11 - Train Loss: 86.0181 - Val L